# Images comparison with GPT-4o Vision

In this lab we are going to use **Azure OprnAI GPT-4o model** to compare two images and describe the difference between them.

You can define where model should focus by proving your instruction in a prompt


> https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models?tabs=python-secure%2Cglobal-standard%2Cstandard-chat-completions#gpt-4o-and-gpt-4-turbo

Import libraries

In [ ]:
import datetime
import openai
import os
import sys

from dotenv import load_dotenv
from IPython.display import Image, HTML
from openai import AzureOpenAI

In [ ]:
def check_openai_version():
    """
    Check Azure Open AI version
    """
    installed_version = openai.__version__

    try:
        version_number = float(installed_version[:3])
    except ValueError:
        print("Invalid OpenAI version format")
        return

    print(f"Installed OpenAI version: {installed_version}")

    if version_number < 1.0:
        print("[Warning] You should upgrade OpenAI to have version >= 1.0.0")
        print("To upgrade, run: %pip install openai --upgrade")
    else:
        print(f"[OK] OpenAI version {installed_version} is >= 1.0.0")


check_openai_version()
print(f"Python version: {sys.version}")

## 1. Azure Open AI

Load environment variables

In [ ]:
load_dotenv(override=True)

# Azure Open AI
openai.api_type: str = "azure"
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")

In [ ]:
model = os.getenv("AZURE_OPENAI_MODEL")

Connect to Azure OpenAI

In [ ]:
endpoint = f"{openai.api_base}/openai/deployments/{model}/chat/completions?api-version={openai.api_version}"

client = AzureOpenAI(
    api_key=openai.api_key,
    api_version=openai.api_version,
    base_url=endpoint,
)

## 2. Images comparison function

- helper function to read and encode image file

In [ ]:
import base64


def open_image_as_base64(filename):
    with open(filename, "rb") as image_file:
        image_data = image_file.read()
    image_base64 = base64.b64encode(image_data).decode("utf-8")
    return f"data:image/png;base64,{image_base64}"

- function to provide image comparison using **Azure OpenAI GPT-4o** model

In [ ]:
def images_comparison(prompt, img_url1, img_url2, imgview=True):
    """
    Images_comparison with Gpt-4o
    """
    print(prompt)
    print()
    
    if imgview:
        # Viewing images
        print(f"Image 1: {img_url1} \nImage 2: {img_url2}")
        display(
            HTML(
                "<table><tr><td><img src={0}></td><td><img src={1}></td></tr></table>".format(
                    img_url1, img_url2
                )
            )
        )

    # Calling Azure OpenAI model
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": open_image_as_base64(img_url1)},
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": open_image_as_base64(img_url2)},
                    },
                ],
            }
        ],
        max_tokens=800,
        temperature=0,
    )

    # Print results
    print()
    print("Azure OpenAI GPT-4o results:")
    print("\033[1;31;34m")
    print(response.choices[0].message.content)

    print("\033[1;31;32m")
    print("[Note] These results are produced by an AI.")

## 3. Testing

Take 2 files: original image and an image generated by **Azure OpenAI Dall-e-3** model

In [ ]:
url1 = "../data/fashion/image1.jpg"
url2 = "../data/artificial_images/dalle3_image1.jpg"

Provide an instruction as prompt 

In [ ]:
prompt = "What are fashion garmets in these images? Is there any difference between them? Explain the details"

In [ ]:
images_comparison(prompt, url1, url2)

## 4. Challenge

1. Compare images in data/fashion folder with artifially generate images in previous lab. They can be found in **data/artificial_images** folder 

In [ ]:
url1 = "..." # Add your image URL
url2 = "..." # Add image URL for comparison

prompt = "..." # Add your prompt
images_comparison(prompt, url1, url2)